# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be passed as an image. Once a request is received, the API will convert load the image preprocess it and pass it to the model. There are two main functions in the API: init() and run(). The init() function loads the model and returns a scoring function. The run() function processes the images and uses the first function to score them.

    Note: Always make sure you don't have any lingering notebooks running (Shutdown previous notebooks). Otherwise it may cause GPU memory issue.

In [12]:
from azureml.core import Workspace
from azureml.core.model import Model
from dotenv import set_key, find_dotenv
import logging
from testing_utilities import get_auth

In [13]:
import keras
import tensorflow

In [14]:
print("Keras: ", keras.__version__)
print("Tensorflow: ", tensorflow.__version__)

Keras:  2.2.0
Tensorflow:  1.10.0


In [15]:
env_path = find_dotenv(raise_error_if_not_found=True)

## Write and save driver script

In [16]:
%%writefile driver.py

from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from azureml.contrib.services.aml_request import rawhttp
from azureml.core.model import Model
from toolz import compose
import numpy as np
import timeit as t
from PIL import Image, ImageOps
import logging

_NUMBER_RESULTS = 3


def _image_ref_to_pil_image(image_ref):
    """ Load image with PIL (RGB)
    """
    return Image.open(image_ref).convert("RGB")


def _pil_to_numpy(pil_image):
    img = ImageOps.fit(pil_image, (224, 224), Image.ANTIALIAS)
    img = image.img_to_array(img)
    return img


def _create_scoring_func():
    """ Initialize ResNet 152 Model
    """
    logger = logging.getLogger("model_driver")
    start = t.default_timer()
    model_name = "resnet_model"
    model_path = Model.get_model_path(model_name)
    model = ResNet152()
    model.load_weights(model_path)
    end = t.default_timer()

    loadTimeMsg = "Model loading time: {0} ms".format(round((end - start) * 1000, 2))
    logger.info(loadTimeMsg)

    def call_model(img_array_list):
        img_array = np.stack(img_array_list)
        img_array = preprocess_input(img_array)
        preds = model.predict(img_array)
        # Converting predictions to float64 since we are able to serialize float64 but not float32
        preds = decode_predictions(preds.astype(np.float64), top=_NUMBER_RESULTS)
        return preds

    return call_model


def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = _create_scoring_func()

    def process_and_score(images_dict):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()
        logger.info("Scoring {} images".format(len(images_dict)))
        transform_input = compose(_pil_to_numpy, _image_ref_to_pil_image)
        transformed_dict = {
            key: transform_input(img_ref) for key, img_ref in images_dict.items()
        }
        preds = scoring_func(list(transformed_dict.values()))
        preds = dict(zip(transformed_dict.keys(), preds))
        end = t.default_timer()

        logger.info("Predictions: {0}".format(preds))
        logger.info("Predictions took {0} ms".format(round((end - start) * 1000, 2)))
        return (preds, "Computed in {0} ms".format(round((end - start) * 1000, 2)))

    return process_and_score


def init():
    """ Initialise the model and scoring function
    """
    global process_and_score
    process_and_score = get_model_api()


@rawhttp
def run(request):
    """ Make a prediction based on the data passed in using the preloaded model
    """
    return process_and_score(request.files)


Overwriting driver.py


## Test the driver¶
We test the driver by passing data.

In [17]:
logging.basicConfig(level=logging.DEBUG)

In [18]:
%run driver.py

Let's load the workspace.

In [ ]:
ws = Workspace.from_config(auth=get_auth())
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

Get the model and score against an example image

In [ ]:
model_path = Model.get_model_path("resnet_model", _workspace=ws)

In [21]:
IMAGEURL = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg"

In [ ]:
# Always make sure you don't have any lingering notebooks running. Otherwise it may cause GPU memory issue.
process_and_score = get_model_api()

In [23]:
resp = process_and_score({"lynx": open("220px-Lynx_lynx_poing.jpg", "rb")})

INFO:model_driver:Scoring 1 images
INFO:model_driver:Predictions: {'lynx': [('n02127052', 'lynx', 0.981648325920105), ('n02128385', 'leopard', 0.007744148373603821), ('n02123159', 'tiger_cat', 0.003686134237796068)]}
INFO:model_driver:Predictions took 3837.34 ms


In [24]:
# Clear GPU memory
from keras import backend as K

In [25]:
K.clear_session()

Next, we will [build a docker image with this modle driver and other supporting files](03_BuildImage.ipynb).